# Agente Inteligente para la Generación Automática de Queries MongoDB a partir de Lenguaje Natural

Este notebook demuestra el uso de un agente inteligente capaz de transformar instrucciones en lenguaje natural en queries MongoDB, incluyendo soporte para joins dinámicos.

In [2]:
import json
with open('../datasets/transactions_collection.json') as f:
    data = json.load(f)
print(f'Total registros: {len(data)}')

Total registros: 8


In [3]:
# Reinicia el kernel antes de ejecutar esta celda para asegurar que los cambios en el archivo fuente se reflejen correctamente.
import sys
sys.path.append('..')
sys.path.append('../src')

import importlib
import AgenteGeneradorQueryMongo
importlib.reload(AgenteGeneradorQueryMongo)
from AgenteGeneradorQueryMongo import SmartMongoQueryGenerator

from dataset_manager import create_default_dataset
manager = create_default_dataset()
generator = SmartMongoQueryGenerator(dataset_manager=manager)

# Ejemplo de instrucción en lenguaje natural para JOIN (proyección dinámica de campos del join)
instruccion_join = "une la colección empleados con la colección departamentos usando el campo departamento_id proyecta los campos nombre, apellido, departamentos_info, departamento_nombre"
query_join = generator.generate_query('empleados', instruccion_join)
print('Query generada para join:')
print(query_join)

Query generada para join:
[
  {
    "$lookup": {
      "from": "departamentos",
      "localField": "departamento_id",
      "foreignField": "departamento_id",
      "as": "departamentos_info"
    }
  },
  {
    "$unwind": "$departamentos_info"
  },
  {
    "$project": {
      "nombre": 1,
      "apellido": 1,
      "departamentos_info": 1,
      "departamento_nombre": 1
    }
  }
]


Puedes modificar la variable `instruccion` para probar otras frases en lenguaje natural, incluyendo joins con diferentes nombres de colección y campo, o instrucciones que no requieran join.

# Ejemplos avanzados de instrucciones en lenguaje natural
A continuación se muestran tres casos de uso para probar el agente generador de queries MongoDB:

In [4]:
# Ejemplo 1: filtra registros cuyo Devices.ServicePoints.ShipOutCycles.Transactions.Total sea mayor a 3000
instruccion = "filtra registros cuyo Total sea mayor a 3000"
query = generator.generate_query('transacciones', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
# Mostrar el pipeline como dict para depuración
import re
pipeline_match = re.search(r'aggregate\\((\[.*\])\\)', query, re.DOTALL)
if pipeline_match:
    print("\nPipeline generado:")
    print(pipeline_match.group(1))

[DEBUG] Pipeline generado: {'$match': {'Devices.ServicePoints.ShipOutCycles.Transactions.Total': {'$gt': 3000.0}}}
Instrucción: filtra registros cuyo Total sea mayor a 3000
Query generado:
[
  {
    "$match": {
      "Devices.ServicePoints.ShipOutCycles.Transactions.Total": {
        "$gt": 3000.0
      }
    }
  }
]


In [5]:
# Ejemplo 2: muestra los nombres y apellidos de los empleados del departamento de ventas
instruccion = "muestra los nombres y apellidos de los empleados del departamento de ventas"
query = generator.generate_query('empleados', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()

Instrucción: muestra los nombres y apellidos de los empleados del departamento de ventas
Query generado:
[
  {
    "$match": {
      "departamento": "ventas"
    }
  },
  {
    "$project": {
      "nombres": 1,
      "departamento": 1,
      "ventas": 1,
      "apellidos": 1,
      "del": 1
    }
  }
]



In [6]:

# Ejemplo 3: cuenta cuántos empleados hay en cada departamento
instruccion = "cuenta cuántos empleados hay en cada departamento"
query = generator.generate_query('empleados', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()

Instrucción: cuenta cuántos empleados hay en cada departamento
Query generado:
[
  {
    "$group": {
      "_id": "$departamento",
      "count": {
        "$sum": 1
      }
    }
  },
  {
    "$project": {
      "departamento": "$_id",
      "count": 1,
      "_id": 0
    }
  }
]



In [7]:

# Ejemplo 4: ordena los empleados por fecha de ingreso descendente
instruccion = "ordena los empleados por fecha de ingreso descendente"
query = generator.generate_query('empleados', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()

Instrucción: ordena los empleados por fecha de ingreso descendente
Query generado:
[
  {
    "$sort": {
      "fechadeingreso": -1
    }
  }
]



In [8]:

# Ejemplo 5: busca empleados cuyo nombre comience con 'A'
instruccion = "busca empleados cuyo nombre comience con 'A'"
query = generator.generate_query('empleados', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()

Instrucción: busca empleados cuyo nombre comience con 'A'
Query generado:
[
  {
    "$match": {
      "nombre": {
        "$regex": "^A",
        "$options": "i"
      }
    }
  }
]



In [9]:
# Ejemplo 6: filtra transacciones realizadas entre 2023-01-01 y 2023-01-31
instruccion = "filtra transacciones realizadas entre 2023-01-01 y 2023-01-31"
query = generator.generate_query('transacciones', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()

Instrucción: filtra transacciones realizadas entre 2023-01-01 y 2023-01-31
Query generado:
[
  {
    "$match": {
      "Date": {
        "$gte": "2023-01-01",
        "$lte": "2023-01-31"
      }
    }
  },
  {
    "$unwind": "$Devices"
  },
  {
    "$unwind": "$Devices.ServicePoints"
  },
  {
    "$unwind": "$Devices.ServicePoints.ShipOutCycles"
  },
  {
    "$unwind": "$Devices.ServicePoints.ShipOutCycles.Transactions"
  }
]



In [10]:
# Ejemplo 7: muestra los 5 productos más vendidos
instruccion = "muestra los 5 productos más vendidos"
query = generator.generate_query('productos', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()

Instrucción: muestra los 5 productos más vendidos
Query generado:
[
  {
    "$group": {
      "_id": "$nombre",
      "total_vendidos": {
        "$sum": "$ventas"
      }
    }
  },
  {
    "$sort": {
      "total_vendidos": -1
    }
  },
  {
    "$limit": 5
  },
  {
    "$project": {
      "producto": "$_id",
      "total_vendidos": 1,
      "_id": 0
    }
  }
]



In [11]:
# Ejemplo 8: agrega la suma total de ventas por mes
instruccion = "agrega la suma total de ventas por mes"
query = generator.generate_query('ventas', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()

Instrucción: agrega la suma total de ventas por mes
Query generado:
[
  {
    "$addFields": {
      "anio_mes": {
        "$substr": [
          "$Date",
          0,
          7
        ]
      }
    }
  },
  {
    "$group": {
      "_id": "$anio_mes",
      "suma_total_ventas": {
        "$sum": "$Devices.ServicePoints.ShipOutCycles.Transactions.Total"
      }
    }
  },
  {
    "$sort": {
      "_id": 1
    }
  },
  {
    "$project": {
      "mes": "$_id",
      "suma_total_ventas": 1,
      "_id": 0
    }
  }
]



In [12]:
# Ejemplo 9: une la colección ventas con clientes usando cliente_id y proyecta nombre_cliente y total_venta
instruccion = "une la colección ventas con clientes usando cliente_id y proyecta nombre_cliente y total_venta"
query = generator.generate_query('ventas', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()

Instrucción: une la colección ventas con clientes usando cliente_id y proyecta nombre_cliente y total_venta
Query generado:
[
  {
    "$lookup": {
      "from": "clientes",
      "localField": "cliente_id y proyecta nombre_cliente y total_venta",
      "foreignField": "cliente_id y proyecta nombre_cliente y total_venta",
      "as": "clientes_info"
    }
  },
  {
    "$unwind": "$clientes_info"
  }
]



In [17]:


#Ejemplo 10: filtra clientes que no hayan realizado compras en el último año
instruccion = "filtra clientes que no hayan realizado compras en el último año"
query = generator.generate_query('clientes', instruccion)
print(f"Instrucción: {instruccion}")
print("Query generado:")
print(query)
print()




Instrucción: filtra clientes que no hayan realizado compras en el último año
Query generado:
[
  {
    "$match": {}
  }
]



In [14]:
# Caso 1: Header simple
instruccion_header = """
crear campo dateMascara que convierta el campo Date a formato %Y%m%d usando los primeros 19 caracteres
crear campo reg que concatene: "1", "002", el campo Date convertido a formato %Y%m%d%H%M%S usando los primeros 19 caracteres, "00", "01", un espacio, un salto de línea, otro salto de línea
"""
print('--- Caso 1: Header ---')
query_header = generator.generate_query('header', instruccion_header)
print(query_header)

--- Caso 1: Header ---
[
  {
    "$project": {
      "_id": 0,
      "dateMascara": {
        "$dateToString": {
          "date": {
            "$dateFromString": {
              "dateString": {
                "$substr": [
                  "$Date",
                  0,
                  19
                ]
              }
            }
          },
          "format": "%Y%m%d"
        }
      },
      "reg": {
        "$concat": [
          "1",
          "002",
          {
            "$dateToString": {
              "date": {
                "$dateFromString": {
                  "dateString": {
                    "$substr": [
                      "$Date",
                      0,
                      19
                    ]
                  }
                }
              },
              "format": "%Y%m%d%H%M%S"
            }
          },
          "00",
          "01",
          " ",
          "\n",
          "\n"
        ]
      }
    }
  }
]


In [15]:
# Caso 2: Detalle avanzado
instruccion_detalle = """
desanidar Devices
desanidar Devices.ServicePoints
desanidar Devices.ServicePoints.ShipOutCycles
desanidar Devices.ServicePoints.ShipOutCycles.Transactions
agrupar por date, deviceId, branchCode, subChannelCode, shipOutCode, currencyCode, confirmationCode y sumar el total de Devices.ServicePoints.ShipOutCycles.Transactions.Total
proyectar los caracteres de la posición 2 en adelante del deviceId
crear campo confirmationCode que sea _id.confirmationCode o " " si es nulo
crear campo totalParteEntera que sea el primer elemento del split del total por punto
crear campo totalParteDecimal que sea el segundo elemento del split del total por punto o "00" si es nulo
crear campo reg que sea la concatenación de "5", la condición de moneda, la fecha, "00", el deviceId con padding, el shipOutCode con padding, la condición de sucursal, el monto con padding y el código de confirmación con padding
ordenar por deviceId y shipOutCode y subChannelCode y currencyCode
"""
print('--- Caso 2: Detalle ---')
query_detalle = generator.generate_query('detalle', instruccion_detalle)
print(query_detalle)

--- Caso 2: Detalle ---
[
  {
    "$unwind": "$Devices"
  },
  {
    "$unwind": "$Devices.ServicePoints"
  },
  {
    "$unwind": "$Devices.ServicePoints.ShipOutCycles"
  },
  {
    "$unwind": "$Devices.ServicePoints.ShipOutCycles.Transactions"
  },
  {
    "$group": {
      "_id": {
        "Date": "$Date",
        "Id": "$Devices.Id",
        "BranchCode": "$Devices.BranchCode",
        "SubChannelCode": "$Devices.ServicePoints.ShipOutCycles.SubChannelCode",
        "Code": "$Devices.ServicePoints.ShipOutCycles.Code",
        "CurrencyCode": "$Devices.ServicePoints.ShipOutCycles.Transactions.CurrencyCode",
        "currencyCode": "$Devices.ServicePoints.ShipOutCycles.Transactions.CurrencyCode"
      },
      "total": {
        "$sum": "$total"
      }
    }
  },
  {
    "$sort": {
      "deviceId": 1,
      "shipOutCode": 1,
      "subChannelCode": 1
    }
  },
  {
    "$project": {
      "deviceId": {
        "$substrCP": [
          "$_id.deviceId",
          2,
          {
        

In [16]:
# Caso 3: Detalle con agrupaciones y padding
instruccion_detalle2 = """
desanidar Devices
desanidar Devices.ServicePoints
desanidar Devices.ServicePoints.ShipOutCycles con preserveNullAndEmptyArrays
desanidar Devices.ServicePoints.ShipOutCycles.Transactions con preserveNullAndEmptyArrays
agrupar por deviceId, branchCode, subChannelCode, shipOutCode, currencyCode y sumar el total de Devices.ServicePoints.ShipOutCycles.Transactions.Total en soles y en dólares según el código de moneda
luego agrupa todo y suma totalSoles y totalDolares, y cuenta total de registros en soles y en dólares según el código de moneda
crear campo totalParteEnteraSoles que sea el primer elemento del split de totalSoles por punto
crear campo totalParteDecimalSoles que sea el segundo elemento del split de totalSoles por punto o "00" si es nulo
crear campo totalParteEnteraDolares que sea el primer elemento del split de totalDolares por punto
crear campo totalParteDecimalDolares que sea el segundo elemento del split de totalDolares por punto o "00" si es nulo
crear campo reg que concatene: "9", el total de registros con padding, el total de registros en soles con padding, el total de registros en dólares con padding, el monto en soles con padding, el monto en dólares con padding, un salto de línea, otro salto de línea
"""
print('--- Caso 3: Detalle agrupado ---')
query_detalle2 = generator.generate_query('detalle2', instruccion_detalle2)
print(query_detalle2)

--- Caso 3: Detalle agrupado ---
[
  {
    "$unwind": "$Devices"
  },
  {
    "$unwind": "$Devices.ServicePoints"
  },
  {
    "$unwind": {
      "path": "$Devices.ServicePoints.ShipOutCycles",
      "preserveNullAndEmptyArrays": true
    }
  },
  {
    "$unwind": {
      "path": "$Devices.ServicePoints.ShipOutCycles.Transactions",
      "preserveNullAndEmptyArrays": true
    }
  },
  {
    "$group": {
      "_id": {
        "Id": "$Devices.Id",
        "BranchCode": "$Devices.BranchCode",
        "SubChannelCode": "$Devices.ServicePoints.ShipOutCycles.SubChannelCode",
        "Code": "$Devices.ServicePoints.ShipOutCycles.Code",
        "currencyCode": "$Devices.ServicePoints.ShipOutCycles.Transactions.CurrencyCode"
      },
      "totalSoles": {
        "$sum": {
          "$cond": [
            {
              "$eq": [
                "$Devices.ServicePoints.ShipOutCycles.Transactions.CurrencyCode",
                "PEN"
              ]
            },
            "$Devices.ServicePoi

# Análisis EDA sobre transactions_collection
A continuación se presentan estadísticas descriptivas y visualizaciones clave para el dataset de transacciones.

In [14]:
# Estadísticas descriptivas de los campos principales
from notebooks.eda_utils import describe_transactions, plot_key_distributions, plot_amount_boxplot

# Detecta la clave que contiene la lista de transacciones
main_list = None
for k, v in data.items():
    if isinstance(v, list) and len(v) > 0 and isinstance(v[0], dict):
        main_list = v
        print(f'Usando la clave "{k}" como lista principal de transacciones.')
        break
if main_list is None:
    raise ValueError('No se encontró una lista de transacciones en el dict principal.')

import pandas as pd
df = pd.DataFrame(main_list)
display(df.describe(include='all'))

Usando la clave "sample_documents" como lista principal de transacciones.


,Date,Devices
count,1,1
unique,1,1
top,2024-01-15T10:30:00Z,"[{'Id': 'DEV001', 'BranchCode': 'PE240', 'Serv..."
freq,1,1


In [15]:
# Visualización de la distribución de montos y campos clave
import sys
sys.path.append('..')
from eda_utils import plot_key_distributions, plot_amount_boxplot
import pandas as pd

# Asegura que data esté definido
try:
    data
except NameError:
    import json
    with open('../datasets/transactions_collection.json') as f:
        data = json.load(f)

# Detecta la lista principal de transacciones si no está definida
try:
    main_list
except NameError:
    main_list = None
    for k, v in data.items():
        if isinstance(v, list) and len(v) > 0 and isinstance(v[0], dict):
            main_list = v
            print(f'Usando la clave "{k}" como lista principal de transacciones.')
            break
    if main_list is None:
        raise ValueError('No se encontró una lista de transacciones en el dict principal.')

# Muestra las primeras claves del DataFrame para ayudar a elegir los campos
sample_df = pd.DataFrame(main_list)
print('Primeras columnas disponibles:', sample_df.columns.tolist())

# Si no hay columnas numéricas, muestra las primeras columnas y grafica las categóricas
numeric_cols = sample_df.select_dtypes(include='number').columns.tolist()
numeric_cols = [col for col in numeric_cols if sample_df[col].dropna().size > 1]
if not numeric_cols:
    print('No hay columnas numéricas con suficientes datos para graficar. Mostrando histogramas de las primeras columnas categóricas.')
    cat_cols = sample_df.select_dtypes(include='object').columns.tolist()
    cat_cols = [col for col in cat_cols if sample_df[col].dropna().size > 1]
    key_fields = cat_cols[:3]
    plot_key_distributions(main_list, key_fields)
else:
    print('Columnas numéricas disponibles:', numeric_cols)
    key_fields = numeric_cols[:3]  # Usa hasta 3 columnas numéricas
    plot_key_distributions(main_list, key_fields)
    plot_amount_boxplot(main_list, key_fields[0])

Primeras columnas disponibles: ['Date', 'Devices']
No hay columnas numéricas con suficientes datos para graficar. Mostrando histogramas de las primeras columnas categóricas.


## ¿Por qué es importante el análisis EDA en este agente?
El análisis exploratorio de datos (EDA) en este notebook no busca construir modelos de machine learning, sino comprender la estructura, calidad y patrones de los datos de transacciones. Esto es fundamental para que el agente inteligente pueda:
- Identificar los campos relevantes y sus tipos para generar queries MongoDB precisos.
- Detectar valores atípicos, distribuciones y relaciones entre campos que pueden influir en la interpretación de instrucciones en lenguaje natural.
- Mejorar el mapeo de sinónimos y la selección de campos clave en las proyecciones y joins.
- Adaptarse a diferentes esquemas de datos y colecciones, permitiendo mayor flexibilidad en la generación automática de queries.
En resumen, el EDA proporciona el contexto necesario para que el agente LLM entienda el dataset y genere consultas más robustas y útiles, alineadas con los principios de los papers académicos sobre agentes inteligentes y procesamiento de lenguaje natural.

In [16]:
# Análisis de valores nulos y outliers en transacciones
from eda_utils import analyze_missing_and_outliers

# Usa la lista principal de transacciones detectada previamente (main_list)
analyze_missing_and_outliers(main_list)

Resumen de valores nulos por columna:
Date       0
Devices    0
dtype: int64
No hay columnas numéricas para analizar outliers.


In [17]:
# Análisis de correlación entre variables numéricas
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df_corr = pd.DataFrame(main_list)
numeric_cols = df_corr.select_dtypes(include='number').columns.tolist()
if len(numeric_cols) > 1:
    corr = df_corr[numeric_cols].corr()
    plt.figure(figsize=(8,6))
    sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title('Matriz de correlación entre variables numéricas')
    plt.show()
else:
    print('No hay suficientes columnas numéricas para mostrar una matriz de correlación.')

No hay suficientes columnas numéricas para mostrar una matriz de correlación.


In [18]:
# Resumen de frecuencias para variables categóricas
import pandas as pd

df_cat = pd.DataFrame(main_list)
cat_cols = df_cat.select_dtypes(include='object').columns.tolist()
if cat_cols:
    for col in cat_cols[:3]:  # Muestra hasta 3 columnas categóricas
        print(f'Frecuencias de valores en {col}:')
        print(df_cat[col].value_counts(dropna=False).head(10))
        print('---')
else:
    print('No hay columnas categóricas para mostrar frecuencias.')

Frecuencias de valores en Date:
Date
2024-01-15T10:30:00Z    1
Name: count, dtype: int64
---
Frecuencias de valores en Devices:
Devices
[{'Id': 'DEV001', 'BranchCode': 'PE240', 'ServicePoints': [{'ShipOutCycles': [{'SubChannelCode': 'CH001', 'Code': 'SO001', 'ConfirmationCode': 'CONF001', 'Transactions': [{'Total': 100.5, 'CurrencyCode': 'PEN'}]}]}]}]    1
Name: count, dtype: int64
---
